In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk

In [38]:
path = '/Users/akhil/AI:ML/Projects/Sentiment_Analysis_Lexican_Based/data/Product_Reviews.csv'

In [39]:
data = pd.read_csv(path, header=0)

In [40]:
data.head()

,Product ID,Product Review,Sentiment
0,AVpe7AsMilAPnD_xQ78G,I initially had trouble deciding between the p...,POSITIVE
1,AVpe7AsMilAPnD_xQ78G,Allow me to preface this with a little history...,POSITIVE
2,AVpe7AsMilAPnD_xQ78G,I am enjoying it so far. Great for reading. Ha...,POSITIVE
3,AVpe7AsMilAPnD_xQ78G,I bought one of the first Paperwhites and have...,POSITIVE
4,AVpe7AsMilAPnD_xQ78G,I have to say upfront - I don't like coroporat...,POSITIVE


In [41]:
import re

In [42]:
a = data['Product Review'][0]

In [43]:
data['review punctuation removed'] = data['Product Review'].apply(lambda x: re.sub('[^A-Za-z]+', ' ', x))

In [44]:
data.rename(columns = {'review punctuation removed': 'cleaned_review'}, inplace=True)

In [45]:
data['cleaned_review'] = data['cleaned_review'].apply(lambda x: x.lower())

In [46]:
from nltk.corpus import stopwords

In [47]:
data['cleaned_review'] = data['cleaned_review'].apply(lambda x: ' '.join([word.strip() for word in x.split() if word not in stopwords.words('english')]))

In [48]:
data['cleaned_review'][0]

'initially trouble deciding paperwhite voyage reviews less said thing paperwhite great spending money go voyage fortunately friends owned ended buying paperwhite basis models ppi dollar jump turns pricey voyage page press always sensitive fine specific setting need auto light adjustment week loving paperwhite regrets touch screen receptive easy use keep light specific setting regardless time day case hard change setting either changing light level certain time day every reading also glad went international shipping option amazon extra expense delivery time tracking didnt need worry customs may used third party shipping service'

In [49]:
nltk.download('wordnet')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package wordnet to /Users/akhil/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [50]:
ps = PorterStemmer()


In [51]:
data['cleaned_review'] = data['cleaned_review'].apply(lambda x : ' '.join([ps.stem(word) for word in x.split()]))
data['cleaned_review'][0]

'initi troubl decid paperwhit voyag review less said thing paperwhit great spend money go voyag fortun friend own end buy paperwhit basi model ppi dollar jump turn pricey voyag page press alway sensit fine specif set need auto light adjust week love paperwhit regret touch screen recept easi use keep light specif set regardless time day case hard chang set either chang light level certain time day everi read also glad went intern ship option amazon extra expens deliveri time track didnt need worri custom may use third parti ship servic'

In [52]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
sentiment = SentimentIntensityAnalyzer()

In [53]:
data['compound_polarity_score'] = data['cleaned_review'].apply(lambda x: sentiment.polarity_scores([x])['compound'])
data.iloc[:3]

,Product ID,Product Review,Sentiment,cleaned_review,compound_polarity_score
0,AVpe7AsMilAPnD_xQ78G,I initially had trouble deciding between the p...,POSITIVE,initi troubl decid paperwhit voyag review less...,0.9423
1,AVpe7AsMilAPnD_xQ78G,Allow me to preface this with a little history...,POSITIVE,allow prefac littl histori casual reader own n...,0.9638
2,AVpe7AsMilAPnD_xQ78G,I am enjoying it so far. Great for reading. Ha...,POSITIVE,enjoy far great read origin fire sinc fire use...,0.0258


In [54]:
def predict_sentiment(score):
    if score >= 0.5: return 'POSITIVE'
    elif score <= -0.5: return 'NEGATIVE'
    else: return 'NEUTRAL'

data['predicted_sentiment'] = data['compound_polarity_score'].apply(predict_sentiment)
data.loc[:3]

,Product ID,Product Review,Sentiment,cleaned_review,compound_polarity_score,predicted_sentiment
0,AVpe7AsMilAPnD_xQ78G,I initially had trouble deciding between the p...,POSITIVE,initi troubl decid paperwhit voyag review less...,0.9423,POSITIVE
1,AVpe7AsMilAPnD_xQ78G,Allow me to preface this with a little history...,POSITIVE,allow prefac littl histori casual reader own n...,0.9638,POSITIVE
2,AVpe7AsMilAPnD_xQ78G,I am enjoying it so far. Great for reading. Ha...,POSITIVE,enjoy far great read origin fire sinc fire use...,0.0258,NEUTRAL
3,AVpe7AsMilAPnD_xQ78G,I bought one of the first Paperwhites and have...,POSITIVE,bought one first paperwhit pleas constant comp...,0.9300,POSITIVE


In [55]:
#Accuracy
(data['Sentiment']==data['predicted_sentiment']).value_counts(normalize=True)

True     0.68719
False    0.31281
Name: proportion, dtype: float64

Accuracy = 69%